In [45]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier


In [12]:
test_features = pd.read_csv('test_features.csv')
train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')


In [24]:
pd.set_option('display.max_columns', 500)

In [35]:
train_features.describe()
# train_features.shape  # 37745, 103
# train_features.head()

,id,member_id,loan_amnt,funded_amnt,installment,annual_inc,url,desc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
count,37745.000000,0.0,37745.000000,37745.000000,37745.000000,3.774500e+04,0.0,0.0,37653.000000,37745.000000,37745.000000,16864.000000,5575.000000,37745.000000,37745.000000,37745.000000,37745.000000,37745.000000,9105.000000,4.738000e+03,4738.000000,37745.000000,3.774500e+04,3.774500e+04,37745.000000,37745.000000,37745.000000,37745.000000,36511.000000,3.774500e+04,31490.000000,37745.000000,37745.000000,37745.000000,37732.000000,3.774500e+04,37745.000000,37745.000000,37745.000000,37745.000000,37742.000000,37102.000000,37078.000000,37745.000000,37745.000000,36511.000000,37745.000000,37745.000000,37745.000000,37745.000000,37148.000000,7750.000000,34599.000000,11171.000000,37745.000000,37745.000000,37745.000000,37745.000000,37745.000000,37745.000000,37745.000000,37745.000000,37745.000000,37745.000000,36735.0,37745.000000,37745.000000,37745.000000,37744.000000,37102.000000,37745.000000,37745.000000,3.774500e+04,3.774500e+04,37745.000000,3.774500e+04,4738.000000,4738.000000,4738.000000,4738.000000,4639.000000,4738.000000,4738.000000,4738.000000,4738.000000,1668.000000
mean,33579.507193,NaN,14913.304411,14913.304411,444.702933,7.962921e+04,NaN,NaN,18.508652,0.236111,0.557186,36.951613,82.125202,11.426917,0.158167,14342.088780,24.199497,0.016823,46.062273,1.256795e+05,18.895046,0.000079,4.891299e+02,1.540845e+05,1.077918,2.637939,0.882183,1.873732,17.787626,3.644567e+04,71.430645,1.381958,2.898874,5272.748655,52.366108,3.796957e+04,1.348232,1.781454,2.411578,5.122162,15076.214006,16452.730715,44.425525,0.008902,1.558908,121.622826,173.994436,13.601802,7.455928,1.541688,23.531684,40.675613,6.498801,37.981559,0.484753,3.280487,4.890025,4.770884,7.440085,8.882077,8.107802,13.569108,4.842019,11.401298,0.0,0.000079,0.064909,2.426070,94.849497,28.945119,0.141052,0.016929,1.955516e+05,5.100947e+04,26270.148496,4.563859e+04,33654.305192,0.724567,1.736809,11.296117,55.261199,2.855424,12.897425,0.048333,0.073660,36.332134
std,13609.172414,NaN,10153.364841,10153.364841,293.571158,8.052291e+04,NaN,NaN,19.670753,0.749590,0.800375,22.029190,23.920019,5.930225,0.407309,21374.848418,12.831247,0.141181,21.649640,6.738376e+04,8.289922,0.008915,4.189616e+04,1.736256e+05,1.218345,2.888993,1.064389,1.741982,23.063966,4.647142e+04,22.591509,1.573718,2.696923,5355.450789,22.432553,3.659178e+04,1.707470,3.011275,2.677461,3.474509,18572.871228,20994.978692,30.114450,0.119502,184.557160,54.482381,97.106759,17.214774,8.311335,1.834105,32.876516,22.249023,5.650770,22.218094,1.379998,2.309597,3.262211,3.241673,4.796524,7.734212,4.945652,8.296456,3.180094,5.921900,0.0,0.008915,0.439450,2.045166,8.681470,34.092375,0.353817,0.191225,1.924783e+

In [18]:
train_features.isna().sum()

id                                         0
member_id                              37745
loan_amnt                                  0
funded_amnt                                0
term                                       0
int_rate                                   0
installment                                0
grade                                      0
sub_grade                                  0
emp_title                               3565
emp_length                              3277
home_ownership                             0
annual_inc                                 0
url                                    37745
desc                                   37745
purpose                                    0
title                                      0
zip_code                                   0
addr_state                                 0
dti                                       92
delinq_2yrs                                0
earliest_cr_line                           0
inq_last_6

In [41]:
# From kaggle, already have data split out into train and test, so can assign directly
# Credit to DMA/LSDS01

X_train = train_features.dropna(axis='columns', how='any')
# X_train = train_features.drop(columns=['id', 'member_id'])
y_train = train_labels.charged_off  # labels are 'id' and 'charged_off'

X_test = test_features.dropna(axis='columns', how='any')
# X_test = test_features.drop(columns=['id', 'member_id'])

In [42]:
X_train.shape, X_test.shape

((37745, 66), (9437, 67))

In [26]:
# Model a Decision Tree regression - HT RH/LSDS

def regress_wave(max_depth):
    tree = DecisionTreeRegressor(max_depth=max_depth)
    tree.fit(X_train, y_train)
    print('Train R^2 score:', tree.score(X_train, y_train))
    # print('Test R^2 score:', tree.score(X_test, y_test))
    plt.scatter(X_train, y_train)
    # plt.scatter(X_test, y_test)
    plt.step(X, tree.predict(X))
    plt.show()


# regress_wave((1,8,1))  # crashes on 'term' feature, which is an object dtype

In [55]:
# Draw from LSDS01 Unit 2 Tanzania waterpump project methods

X_train_no_object_dtypes = X_train.select_dtypes(include='number')
# X_train_no_object_dtypes = X_train_no_object_dtypes.dropna()

X_test_no_object_dtypes = X_test.select_dtypes(include='number')

model = DecisionTreeClassifier()
# model.get_params().keys()

param_grid = {}
# param_grid = {'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5, 6, 7, 8]}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=2)  # can define specific args here

grid_search.fit(X_train_no_object_dtypes, y_train)
# estimator is your model or pipeline, which you've instantiated and fitted

# X_test is your dataframe or numpy array, 
# with the same number of rows, in the same order, as test_features, 
# and the same number of columns, in the same order, as X_train

# y_pred = grid_search.predict(X_test_no_object_dtypes)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=2, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [56]:
# Show best parameters

print(f'Best parameters: {grid_search.best_params_}')
print(f'Best score: {grid_search.best_score_:0.4f}')

Best parameters: {}
Best score: 0.7550


In [60]:
# Submit predictive probabilities to kaggle

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['charged_off'] = grid_search.predict_proba(X_test_no_object_dtypes)[:, 1]
submission.to_csv('submission-001.csv', index=False)

ValueError: Number of features of the model must match the input. Model n_features is 53 and input n_features is 54 